In [ ]:
import tensorflow as tf
from tensorflow.keras import optimizers
from tensorflow.keras.models import Sequential
from tensorflow.keras import models
from tensorflow.keras.regularizers import l2
from tensorflow.keras.callbacks import ReduceLROnPlateau, EarlyStopping, ModelCheckpoint
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Activation, Dropout, Flatten, Dense, BatchNormalization
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
df = pd.read_csv("data/icml_face_data.csv")

In [ ]:
df.columns = ['emotion', 'Usage', 'pixels']

In [ ]:
train = df.loc[df.Usage == 'Training']
train2 = df.loc[df.Usage == 'PublicTest']
test = df.loc[df.Usage == 'PrivateTest']

In [ ]:
xtrain = []
ytrain = []

for i in range(len(train)):
    label = train.iloc[i][0]
    img = np.array([int(x) for x in train.iloc[i][2].split(' ')]).astype('float32')
    img = img/255
    img = img.reshape(48, 48, 1)
    xtrain.append(img)
    ytrain.append(label)

for i in range(len(train2)):
    label = train2.iloc[i][0]
    img = np.array([int(x) for x in train2.iloc[i][2].split(' ')]).astype('float32')
    img = img/255
    img = img.reshape(48, 48, 1)
    xtrain.append(img)
    ytrain.append(label)
    
xtest = []
ytest = []

for i in range(len(test)):
    label = test.iloc[i][0]
    img = np.array([int(x) for x in test.iloc[i][2].split(' ')]).astype('float32')
    img = img/255
    img = img.reshape(48, 48, 1)
    xtest.append(img)
    ytest.append(label)

In [ ]:
ytrain = tf.keras.utils.to_categorical(ytrain, num_classes=7)
ytest = tf.keras.utils.to_categorical(ytest, num_classes=7)

In [ ]:
xtrain = tf.convert_to_tensor(xtrain)
ytrain = tf.convert_to_tensor(ytrain)
xtest = tf.convert_to_tensor(xtest)
ytest = tf.convert_to_tensor(ytest)

In [ ]:
model = Sequential()
model.add(Conv2D(64, (3, 3), activation='relu', input_shape=(48, 48, 1), kernel_regularizer=l2(0.01)))
model.add(Conv2D(64, (3, 3), padding='same',activation='relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2), strides=(2, 2)))
model.add(Dropout(0.5))    
model.add(Conv2D(64, (3, 3), padding='same', activation='relu'))
model.add(BatchNormalization())
model.add(Conv2D(64, (3, 3), padding='same', activation='relu'))
model.add(BatchNormalization())
model.add(Conv2D(64, (3, 3), padding='same', activation='relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.5))
model.add(Conv2D(128, (3, 3), padding='same', activation='relu'))
model.add(BatchNormalization())
model.add(Conv2D(128, (3, 3), padding='same', activation='relu'))
model.add(BatchNormalization())
model.add(Conv2D(128, (3, 3), padding='same', activation='relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.5))    
model.add(Conv2D(256, (3, 3), padding='same', activation='relu'))
model.add(BatchNormalization())
model.add(Conv2D(256, (3, 3), padding='same', activation='relu'))
model.add(BatchNormalization())
model.add(Conv2D(256, (3, 3), padding='same', activation='relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.5))    
model.add(Conv2D(512, (3, 3), padding='same', activation='relu'))
model.add(BatchNormalization())
model.add(Conv2D(512, (3, 3), padding='same', activation='relu'))
model.add(BatchNormalization())
model.add(Conv2D(512, (3, 3), padding='same', activation='relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.5))    
model.add(Flatten())
model.add(Dense(512, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(7, activation='softmax'))

In [ ]:
model.summary()

In [ ]:
lr = 0.001
adam = optimizers.Adam(learning_rate = lr)
model.compile(optimizer=adam,
             loss='categorical_crossentropy',
             metrics=['accuracy'])

In [ ]:
lr_reducer = ReduceLROnPlateau(monitor='loss', factor=0.9, patience=3)
early_stopper = EarlyStopping(monitor='accuracy', min_delta=0, patience=6, mode='auto')
checkpointer = ModelCheckpoint('Model/weights.hd5', monitor='loss', verbose=1, save_best_only=True)

In [ ]:
model.load_weights('Model/weights.hd5')

In [ ]:
model.fit(x=xtrain, 
          y=ytrain, 
          epochs=150, 
          batch_size=64, 
          shuffle=True, 
          callbacks=[lr_reducer, checkpointer, early_stopper] )

In [ ]:
model.save('models/model.h5')

In [ ]:
model.evaluate(xtest, ytest, batch_size=64, verbose=1)